In [37]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [38]:
  from google.colab import drive
  drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
dataset_path = '/content/drive/My Drive/data_capstone'
wisata_data = pd.read_csv('/content/drive/My Drive/data_capstone/cleaned_dataset_wisata.csv')
# user_data = pd.read_csv('/content/drive/My Drive/data_capstone/user.csv')



In [40]:
wisata_data.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Coordinate', 'Lat', 'Long', 'Rating_Count'],
      dtype='object')

In [41]:
# Cek data yang  di setiap kolom
wisata_data.isnull().sum()


Place_Id        0
Place_Name      0
Description     0
Category        0
City            0
Price           0
Rating          0
Coordinate      0
Lat             0
Long            0
Rating_Count    0
dtype: int64

In [42]:
print(wisata_data.shape)

(437, 11)


In [43]:
# Cek duplikat pada dataset
duplicate_rows = wisata_data[wisata_data.duplicated()]
print("Jumlah baris duplikat:", duplicate_rows.shape[0])
print("Baris duplikat:")
print(duplicate_rows)


Jumlah baris duplikat: 0
Baris duplikat:
Empty DataFrame
Columns: [Place_Id, Place_Name, Description, Category, City, Price, Rating, Coordinate, Lat, Long, Rating_Count]
Index: []


Pembuatan TF-IDF Matrix

In [44]:

wisata_data['City_Category'] = wisata_data['City'] + ' ' + wisata_data['Category']


wisata_data['City_Category'] = wisata_data['City_Category'].fillna('')


tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(wisata_data['City_Category'])

tfidf_matrix.shape


(437, 15)

In [45]:

wisata_data['Rating'] = wisata_data['Rating'].fillna(wisata_data['Rating'].mean())

tfidf_matrix = tfidf.fit_transform(wisata_data['Category'] + ' ' + wisata_data['City'] + ' ' + wisata_data['Rating'].astype(str))


In [46]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [47]:

indices = pd.Series(wisata_data.index, index=wisata_data['Place_Name']).drop_duplicates()

Fungsi Rekomendasi

In [48]:
def get_recommendations(Category, City, cosine_sim=cosine_sim):
    filtered_data =  wisata_data[( wisata_data['Category'] == Category) & ( wisata_data['City'] == City)]

    indices = filtered_data.index.tolist()

    similarity_scores = [(idx, cosine_sim[idx]) for idx in indices]

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    similarity_scores = similarity_scores[1:16]

    similar_places_indices = [i[0] for i in similarity_scores]

    similar_places_indices = [i for i in similar_places_indices if  wisata_data.index[i] not in indices]

    return  wisata_data[['Place_Name', 'City', 'Rating']].iloc[similar_places_indices]


Model dan Latih Model

In [49]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(wisata_data, test_size=0.2, random_state=42)

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):

    if(logs.get('accuracy') > 0.98):

      print("\naccuracy is higher than 0.98 so canceling training!")
      self.model.stop_training = True

callbacks = myCallback()

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(tfidf_matrix.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=['accuracy'])

model.fit(tfidf_matrix.toarray(), tfidf_matrix.toarray(), epochs=50, batch_size=32, validation_split=0.2, callbacks=[callbacks])



Epoch 1/50
11/11 [==============================] - 3s 55ms/step - loss: 0.1826 - accuracy: 0.8534 - val_loss: 0.1432 - val_accuracy: 0.8653
Epoch 2/50
11/11 [==============================] - 0s 16ms/step - loss: 0.1039 - accuracy: 0.8682 - val_loss: 0.0635 - val_accuracy: 0.8653
Epoch 3/50
11/11 [==============================] - 0s 13ms/step - loss: 0.0421 - accuracy: 0.8682 - val_loss: 0.0331 - val_accuracy: 0.8653
Epoch 4/50
11/11 [==============================] - 0s 11ms/step - loss: 0.0328 - accuracy: 0.8682 - val_loss: 0.0326 - val_accuracy: 0.8653
Epoch 5/50
11/11 [==============================] - 0s 17ms/step - loss: 0.0335 - accuracy: 0.8682 - val_loss: 0.0328 - val_accuracy: 0.8653
Epoch 6/50
11/11 [==============================] - 0s 15ms/step - loss: 0.0334 - accuracy: 0.8682 - val_loss: 0.0325 - val_accuracy: 0.8653
Epoch 7/50
11/11 [==============================] - 0s 14ms/step - loss: 0.0329 - accuracy: 0.8682 - val_loss: 0.0323 - val_accuracy: 0.8653
Epoch 8/50
11

In [50]:
loss, accuracy = model.evaluate(tfidf_matrix.toarray(), tfidf_matrix.toarray())
accuracy_percent = accuracy * 100
print(f'Model Loss: {loss}, Model Accuracy: {accuracy_percent:.2f}%')


14/14 [==============================] - 0s 3ms/step - loss: 0.0323 - accuracy: 0.8676
Model Loss: 0.03230658173561096, Model Accuracy: 86.76%


In [51]:
filtered_data = wisata_data[(wisata_data['Category'] == 'Taman Hiburan') & (wisata_data['City'] == 'Jakarta')]
selected_columns = filtered_data[['Place_Id', 'Place_Name', 'Category', 'City', 'Coordinate']].head(7)
print(selected_columns)

    Place_Id                         Place_Name       Category     City  \
2          3                      Dunia Fantasi  Taman Hiburan  Jakarta   
3          4  Taman Mini Indonesia Indah (TMII)  Taman Hiburan  Jakarta   
4          5           Atlantis Water Adventure  Taman Hiburan  Jakarta   
5          6            Taman Impian Jaya Ancol  Taman Hiburan  Jakarta   
7          8                      Ocean Ecopark  Taman Hiburan  Jakarta   
18        19                           Kidzania  Taman Hiburan  Jakarta   
26        27                          Sea World  Taman Hiburan  Jakarta   

                                         Coordinate  
2   {'lat': -6.125312399999999, 'lng': 106.8335377}  
3   {'lat': -6.302445899999999, 'lng': 106.8951559}  
4              {'lat': -6.12419, 'lng': 106.839134}  
5   {'lat': -6.117333200000001, 'lng': 106.8579951}  
7   {'lat': -6.125801699999999, 'lng': 106.8363249}  
18  {'lat': -6.225073499999999, 'lng': 106.8097144}  
26   {'lat': -6.12647

In [3]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('saved_model/modelrekomen')

NameError: ignored